# Documentation du Projet Poppy Torso

Ce document décrit les fonctionnalités et le code d'un projet utilisant le robot Poppy Torso. Le projet inclut des séquences de mouvement, de la détection d'objets, et des interactions vocales via des modèles d'IA.

## **1. Prérequis**
- **Matériel** :
  - Robot Poppy Torso (ou simulateur V-REP).
  - Caméra USB (pour la détection de couleurs).
  - Microphone (pour les commandes vocales).
- **Logiciel** :
  - Environnement Python 3.8+.
  - Librairies : `pypot`, `numpy`, `opencv-python`, `speech_recognition`, `gTTS`, `transformers`, `openai`, etc.
  - Simulateur V-REP (si utilisé).
  - Clés API pour OpenAI et Mistral (pour les assistants vocaux).


## **2. Fonctionnalités principales**
### **2.1 Séquence de salutation**
- **Description** : Le robot lève les bras, salue, et incline son torse.
- **Fichier** : `document2.ipynb` (première cellule).
- **Fonctions clés** :
  - `smooth_move()` : Interpolation de mouvement avec une courbe de Bézier.
  - `salutation_sequence()` : Orchestre les étapes de la salutation.

In [ ]:
from pypot.creatures import PoppyTorso
import time
import numpy as np

# Initialisation du robot Poppy Torso
robot = PoppyTorso(simulator="vrep")

# Définition des moteurs principaux
motors = {
    "torso": robot.bust_y,
    "left_shoulder": robot.l_shoulder_y,
    "right_shoulder": robot.r_shoulder_y,
    "left_arm": robot.l_arm_z,
    "right_arm": robot.r_arm_z
}

# Fonction pour interpoler en douceur les mouvements avec une courbe de Bézier
def smooth_move(motor, start, end, duration=2, steps=50):
    t = np.linspace(0, 1, steps)
    bezier = (1 - t) ** 2 * start + 2 * (1 - t) * t * ((start + end) / 2) + t ** 2 * end
    for pos in bezier:
        motor.goal_position = pos
        time.sleep(duration / steps)

# Réinitialisation du robot en position neutre
def reset_position():
    for motor in motors.values():
        motor.goal_position = 0
    time.sleep(1)

# Séquence de salutation et inclinaison
def salutation_sequence():
    print("Début de la salutation...")

    # Lever les bras
    smooth_move(motors["left_shoulder"], 0, 40, duration=1)
    smooth_move(motors["right_shoulder"], 0, -40, duration=1)

    # Saluer avec un bras
    smooth_move(motors["left_arm"], 0, 30, duration=1)
    smooth_move(motors["left_arm"], 30, 0, duration=1)

    # Inclinaison du torse
    smooth_move(motors["torso"], 0, 20, duration=1.5)
    smooth_move(motors["torso"], 20, -10, duration=1)
    smooth_move(motors["torso"], -10, 0, duration=1)

    # Repose les bras
    smooth_move(motors["left_shoulder"], 40, 0, duration=1)
    smooth_move(motors["right_shoulder"], -40, 0, duration=1)

    print("Salutation terminée.")

# Lancer le mouvement
reset_position()
salutation_sequence()
reset_position()

print("Fin du programme.")
robot.close()

### **2.2 Mouvement "Dab"**
- **Description** : Le robot effectue un mouvement de dab.
- **Fonctions clés** :
  - `dab_move()` : Contrôle les épaules, coudes, et tête pour réaliser le dab.

In [ ]:
from pypot.creatures import PoppyTorso
import time
import numpy as np
robot = PoppyTorso(simulator='vrep')
# Définition des moteurs principaux
motors = {
    "head_z": robot.head_z,
    "head_y": robot.head_y,
    "torso": robot.bust_y,
    "left_arm": robot.l_arm_z,
    "right_arm": robot.r_arm_z,
    "right_elbow": robot.r_elbow_y,
    "left_elbow": robot.l_elbow_y,
    "left_shoulder_y": robot.l_shoulder_y,
    "right_shoulder_y": robot.r_shoulder_y,
    "left_shoulder_x" : robot.l_shoulder_x,
    "right_shoulder_x": robot.r_shoulder_x
}
# Fonction de déplacement en douceur
def smooth_move (motor,target_position,duration=1.5):
    initial_position= motor.present_position
    duration=duration/2
    steps =50
    for i in range (steps):
        motor.goal_position= initial_position+ (target_position-initial_position)*(i/steps)
        time.sleep(duration/steps)
        
# Réinitialisation du robot en position neutre
def reset_position():
    for motor in motors.values():
        smooth_move(motor,0,1.5)
    time.sleep(1)
    
# Fonction de faire le dab
def dab_move():
    print("Exécution du dab")
    
    #lever le bras droit vers le haut 
    smooth_move(motors["right_arm"],-90,duration=1.5)
    
    
    #lever l'épaule droite
    smooth_move(motors["left_shoulder_y"],-90,duration=1.5)
    smooth_move(motors["left_elbow"],-30,duration=1.5)

    
    #lever l'épaule gauche
    smooth_move(motors["right_shoulder_y"],-90,duration=1.5)
    
    
    #Incliner la tête vers le bas
    smooth_move(motors["head_z"],30,duration=1)
    smooth_move(motors["head_y"],30,duration=1)

    # tendre le bras
    smooth_move(motors["right_elbow"],90,duration=1.5)
    smooth_move(motors["right_shoulder_x"],-90,duration=1.5)

    #baisser le bras gauche en diagonale
    smooth_move(motors["left_arm"],-90,duration=1.5)
    #Inciner le torse 
    smooth_move(motors["torso"],15,duration=1)
    
    print("dab éffectué")
    
    #maintenir la position pendant 5 secondes
    time.sleep(5)
    
    #retour à la position normale
    reset_position()
    
#lancer le mouvement
reset_position()
dab_move()
reset_position()

print("Fin du programme")
robot.close()

### Déplacement d'un objet

In [ ]:
from pypot.creatures import PoppyTorso
import time
import numpy as np
import threading

robot = PoppyTorso(simulator="vrep")

# Définition des moteurs principaux
motors = {
    "head_z": robot.head_z,
    "head_y": robot.head_y,
    "torso": robot.bust_y,
    "left_shoulder_y": robot.l_shoulder_y,
    "right_shoulder_y": robot.r_shoulder_y,
    "left_shoulder_x" : robot.l_shoulder_x,
    "right_shoulder_x": robot.r_shoulder_x,
    "left_arm": robot.l_arm_z,
    "right_arm": robot.r_arm_z,
    "right_elbow": robot.r_elbow_y,
    "left_elbow": robot.l_elbow_y,
    "abs_z": robot.abs_z
}
for i in motors.values():
    i.speed_limit =20
# Réinitialisation du robot en position neutre
def reset_position():
    for motor in motors.values():
        motor.goal_position=0
    time.sleep(1)
# fonction de ramassage d'objet

def soulever():
    print("début du levage")
    #position initiale bras ouverts
    T1=threading.Thread(target=smooth_move,args=(motors["left_arm"],30,1.5))
    T2=threading.Thread(target=smooth_move,args=(motors["right_arm"],-30,1.5))
    T1.start()
    T2.start()
    T1.join()
    T2.join()
    time.sleep(2)
    #descente des bras pour attraper l'objet
    T1=threading.Thread(target=smooth_move,args=(motors["left_shoulder_y"],15,1.5))
    T2=threading.Thread(target=smooth_move,args=(motors["right_shoulder_y"],15,1.5))
    T1.start()
    T2.start()
    T1.join()
    T2.join()
    time.sleep(2)
    
    #fermeture des bras(prise)
    T1=threading.Thread(target=smooth_move,args=(motors["left_arm"],-20,1.5))
    T2=threading.Thread(target=smooth_move,args=(motors["right_arm"],20,1.5))
    T1.start()
    T2.start()
    T1.join()
    T2.join()
    time.sleep(2)
    
    #remontée de l'objet
    T1=threading.Thread(target=smooth_move,args=(motors["left_shoulder_y"],-45,1.5))
    T2=threading.Thread(target=smooth_move,args=(motors["right_shoulder_y"],-45,1.5))
    T1.start()
    T2.start()
    T1.join()
    T2.join()
    time.sleep(2)
    time.sleep(2)
        
    print('objet soulevé avec succès')
    
def deplacer():
    print("début du déplacement de l'objet")
    smooth_move(motors["abs_z"],90,duration=1.5)
    time.sleep(2)
    
def deposer():
    print("début du dépot de l'objet")
    T1=threading.Thread(target=smooth_move,args=(motors["left_shoulder_y"],-10,1.5))
    T2=threading.Thread(target=smooth_move,args=(motors["right_shoulder_y"],-10,1.5))
    T1.start()
    T2.start()
    T1.join()
    T2.join()
    time.sleep(2)
    time.sleep(2)
    
    T1=threading.Thread(target=smooth_move,args=(motors["left_arm"],0,1.5))
    T2=threading.Thread(target=smooth_move,args=(motors["right_arm"],0,1.5))
    T1.start()
    T2.start()
    T1.join()
    T2.join()
    time.sleep(2)
        
#lancer le mouvement
reset_position()
soulever()
deplacer()
deposer()
reset_position()

robot.close()

### **2.4 Tri d'objets par couleur**
- **Description** : Détection de couleur via caméra et déplacement d'objets.
- **Fonctions clés** :
  - `detecter_couleur()` : Analyse la région centrale de l'image en HSV.
  - `soulever()`, `deplacer()`, `deposer()` : Contrôlent les bras pour manipuler les objets.

In [ ]:
import cv2
import numpy as np
from pypot.creatures import PoppyTorso
import time

# ===============================
# 1. Initialisation du robot
# ===============================
poppy = PoppyTorso(simulator='vrep')
print("Moteurs disponibles :", [m.name for m in poppy.motors])

# ===============================
# 2. Définition des positions des bacs
# ===============================
POSITIONS = {
    "rouge": {  # Bac 1 (72°)
        "abs_z": 72, "r_shoulder_y": -30, "r_shoulder_x": 15, "r_elbow_y": -20,
    },
    "vert": {  # Bac 2 (144°)
        "abs_z": 144, "r_shoulder_y": -15, "r_shoulder_x": 10, "r_elbow_y": -10,
    },
    "bleu": {  # Bac 3 (216°)
        "abs_z": 216, "r_shoulder_y": 10, "r_shoulder_x": -10, "r_elbow_y": 5,
    },
    "jaune": {  # Bac 4 (288°)
        "abs_z": 288, "r_shoulder_y": 20, "r_shoulder_x": -15, "r_elbow_y": 10,
    },
    "violet": {  # Bac 5 (360° ou 0°)
        "abs_z": 0, "r_shoulder_y": 25, "r_shoulder_x": -20, "r_elbow_y": 15,
    },
}

# 3. Définition des plages de couleurs en HSV
COULEURS = {
    "rouge": [(0, 120, 70), (10, 255, 255)],
    "vert":  [(40, 40, 40), (80, 255, 255)],
    "bleu":  [(100, 150, 0), (140, 255, 255)],
    "jaune": [(20, 100, 100), (30, 255, 255)],
    "violet": [(130, 50, 50), (160, 255, 255)],
}

# 4. Initialisation de la caméra
cap = cv2.VideoCapture(0)

def detecter_couleur(image):
    """
    Détecte la couleur dominante dans la région centrale de l'image.
    Retourne une des clés de COULEURS si la détection est positive, sinon "inconnu".
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    hauteur, largeur, _ = image.shape
    x_start, x_end = largeur // 3, 2 * largeur // 3
    y_start, y_end = hauteur // 3, 2 * hauteur // 3
    roi = hsv[y_start:y_end, x_start:x_end]

    for couleur, (borne_basse, borne_haute) in COULEURS.items():
        masque = cv2.inRange(roi, np.array(borne_basse), np.array(borne_haute))
        if cv2.countNonZero(masque) > 500:
            return couleur
    return "inconnu"

try:
    start_time = time.time()
    timeout = 10  # Durée maximale de détection en secondes
    couleur_trouvee = None

    # Boucle de détection pendant 10 secondes max
    while (time.time() - start_time) < timeout:
        ret, frame = cap.read()
        if not ret:
            print("Erreur de capture vidéo.")
            break

        couleur = detecter_couleur(frame)
        print(f"Couleur détectée : {couleur}")

        if couleur in POSITIONS:
            couleur_trouvee = couleur
            break  # Sortie anticipée si couleur détectée

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if couleur_trouvee:
        print(f"Traitement de la couleur {couleur_trouvee}...")
        positions_cibles = POSITIONS[couleur_trouvee]
        
        # 1. Tourner vers la position du bac
        poppy.abs_z.goto_position(positions_cibles["abs_z"], duration=2, wait=True)

        # 2. Déplacer le bras pour déposer l'objet
        for motor_name, angle in positions_cibles.items():
            if motor_name != "abs_z":
                motor = getattr(poppy, motor_name)
                motor.goto_position(angle, duration=2, wait=True)

        # 3. Revenir à la position initiale
        poppy.abs_z.goto_position(0, duration=2, wait=True)
        poppy.r_shoulder_y.goto_position(0, duration=2, wait=True)
        poppy.r_shoulder_x.goto_position(0, duration=2, wait=True)
        poppy.r_elbow_y.goto_position(0, duration=2, wait=True)
    else:
        print("Aucune couleur détectée après 10 secondes. Arrêt du programme.")

finally:
    cap.release()
    cv2.destroyAllWindows()
    poppy.close()

### **2.5 Assistants vocaux**
- **Assistants disponibles** :
  - **OpenAI GPT-3.5** : Intégration avec l'API OpenAI pour des réponses génératives.
  - **Mistral-7B** : Modèle local via `transformers` pour des réponses hors ligne.
- **Fonctionnalités** :
  - Activation par mot-clé ("Poppy").
  - Synthèse vocale avec `gTTS`.

### 1. Vosk (modèle local)

In [ ]:
# import speech_recognition as sr
from vosk import Model, KaldiRecognizer
import pyaudio
import pyttsx3
from pypot.creatures import PoppyTorso
import os

# Configuration Vosk
MODEL_PATH = os.path.join(os.path.dirname(__file__), "vosk-model-small-fr-0.22")
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("Téléchargez le modèle depuis https://alphacephei.com/vosk/models")

model = Model(MODEL_PATH)
recognizer = KaldiRecognizer(model, 16000)

engine = pyttsx3.init()

robot = PoppyTorso(simulator='vrep')

def écouter():
    mic = pyaudio.PyAudio()
    stream = mic.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8192)
    print("Parlez...")
    stream.start_stream()
    while True:
        data = stream.read(4096)
        if recognizer.AcceptWaveform(data):
            texte = recognizer.Result()[14:-3]
            return texte

def parler(texte):
    engine.say(texte)
    engine.runAndWait()

while True:
    commande = écouter()
    print("Commande détectée :", commande)
    if "poppy" in commande.lower():
        # Remplacez cette partie par votre logique locale (ex : règles prédéfinies)
        reponse = "Bonjour ! Je suis Poppy, votre assistant hors ligne."
        parler(reponse)
        
        if "danse" in commande.lower():
            robot.play_sync("danse_1")

### 2. Openai

In [ ]:
from openai import OpenAI
import speech_recognition as sr
from gtts import gTTS
import os
import time

#config initiale
client= OpenAI(api_key ="your-key") #ajouter une clé api dans les variables d'environnement

WAKE_WORD= "poppy" #mot d'activation
Langue="fr" #fr en francais, en pour anglais

def ecouter_commande(): 
    """ecouter et retranscrire la voix de l'utilisateur"""
    recognizer=sr.Recognizer()

    with sr.Microphone() as source: 
        print("ecoute...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try: 
        texte=recognizer.recognize_google(audio, language= Langue+ "-FR")
        print(f"vous:{texte}")
        return texte.lower()

    except sr.UnknownValueError:
        return ""
    except sr.RequestError:
        print('Service de reconnaissance vocale indisponible')
        return ""

def generer_reponse(prompt):
    """Genere une reponse avec chatgpt"""
    try:
        reponse=client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role":"system","content":"Vous êtes un assistant utile. Repondez de manière concise."},{"role":"user","content":prompt}])
        return response.choices[0].message['content'].strip()

    except Exception as e:
        return f"Erreur: {str(e)}"


def parler(texte):
    """convertir le texte en parole"""
    tts=gTTS (text=texte, lang = Langue, slow= False)
    tts.save('response.mp3')
    os.system("mpg321 response.mp3" if os.name=="posix" else "start response.mp3")

def assistant_vocal():
    print(f"Assistant prêt, Dites ' {WAKE_WORD} ' pour activer")

    while True:
        texte=ecouter_commande()

        if texte and WAKE_WORD in texte: 
            parler ("Oui, comment puis-je vous aider")
            print("En ecoute")
            commande=ecouter_commande()

            if commande:
                response = generer_reponse(commande)
                print(f"IA:{response}")
                parler(response)

            else:
                parler ("Je nai pas compris votre demande")

        elif texte and "arrête" in texte: 
            parler ("Au revoir")
            break
if __name__=="__main__":
    assistant_vocal()


### 3.Mistral-AI

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import speech_recognition as sr
from gtts import gTTS
import os
import time
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
Mistral_API_KEY="your_key"
client=MistralClient(api_key=Mistral_API_KEY)

#charger le modèle (7B paramètres)
model_name="mistralai/Mistral-7B-Instruct-v0.3"
tokenizer= AutoTokenizer.from_pretrained(model_name)
model= AutoModelForCausalLM.from_pretrained(model_name)

#creation pipeline de génération
mistral_pipeline=pipeline("text generation", model=model, tokenizer = tokenizer, max_new_tokens=256, temparature=0.7)

WAKE_WORD= "poppy" #mot d'activation
Langue="fr" #fr en francais, en pour anglais

def ecouter_commande(): 
    """ecouter et retranscrire la voix de l'utilisateur"""
    recognizer=sr.Recognizer()

    with sr.Microphone() as source: 
        print("ecoute...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try: 
        texte=recognizer.recognize_google(audio, language= Langue+ "-FR")
        print(f"vous:{texte}")
        return texte.lower()

    except sr.UnknownValueError:
        return ""
    except sr.RequestError:
        print('Service de reconnaissance vocale indisponible')
        return ""

def generer_reponse(prompt):
    """Genere une reponse avec Mistral"""
    system_prompt="<s> [INST] Vous êtes un assistant utile. Repondez en français de manière concise. [/INST]"
    full_prompt=f"{system_prompt}{prompt}</s>"
    try:
        reponse=mistral_pipeline(
            full_prompt,pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
            
        return response.split("</s>")[-1].strip()

    except Exception as e:
        return f"Erreur: {str(e)}"


def parler(texte):
    """convertir le texte en parole"""
    tts=gTTS (text=texte, lang = Langue, slow= False)
    tts.save('response.mp3')
    os.system("mpg321 response.mp3" if os.name=="posix" else "start response.mp3")

def assistant_vocal():
    print(f"Assistant Mistral prêt, Dites ' {WAKE_WORD} ' pour activer")

    while True:
        texte=ecouter_commande()

        if texte and WAKE_WORD in texte: 
            parler ("Oui, comment puis-je vous aider")
            print("En ecoute")
            commande=ecouter_commande()

            if commande:
                response = generer_reponse(commande)
                print(f"Mistral:{response}")
                parler(response)

            else:
                parler ("Je nai pas compris votre demande")

        elif texte and "arrête" in texte: 
            parler ("Au revoir")
            break
if __name__=="__main__":
    assistant_vocal()
